In [1]:
import os
import sys
import numpy
import random
from time import sleep
from copy import deepcopy
import tensorflow.keras as keras
from keras import layers
from itertools import count
from gui.puzzle_class import Twisty_Puzzle
from gui.ai_modules.ai_data_preparation import state_for_ai
from gui.ai_modules.ai_puzzle_class import puzzle_ai
from gui.ai_modules.twisty_puzzle_model import *

def compute_action_map (puzzle):
    return list(puzzle.moves.keys())

def action_num (action_map, action):
    for i, act in enumerate(action_map):
        if action == act :
            return i

def action_from_num (action_map, num):
    action_map[num]

def prepare_data (action_map, q_table):
    training_table = dict()
    output_template = [0 for _ in range(len(action_map))]
    for state_action , value in q_table.items() :
        (state , action) = state_action
        inner = training_table.get(state) or output_template.copy()
        inner[action_num(action_map, action)] = value
        training_table[state] = inner
    return list(training_table.keys()) , list(training_table.values())

def import_q_table (filename):
    q_table = dict()
    with open(filename, "r") as file:
        q_table = eval (file.read())
    return q_table

<IPython.core.display.Javascript object>

In [2]:
def initialize_nn (puzzle):
    action_map = compute_action_map(puzzle)
    solved_state = state_for_ai(puzzle.SOLVED_STATE)[0]
    input_size = len(solved_state)
    output_size = len(action_map)
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_size,)))
    model.add(layers.Dense(input_size, activation='tanh'))
    model.add(layers.Dense((input_size+output_size)/2, activation='tanh'))
    model.add(layers.Dense(output_size, activation='tanh'))
    model.add(layers.Dense(output_size, activation='tanh'))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model, action_map

In [3]:
def train_nn (model, samples, labels, epochs=100, batch_size=30):
    return model.fit(samples, labels, epochs=epochs, batch_size=batch_size,
                     use_multiprocessing=True, verbose=True)

def prepare_nn (puzzle_name, epochs, batch_size=100):
    puzzle = Twisty_Puzzle()
    puzzle.load_puzzle(puzzle_name)
    (model, action_map) = initialize_nn(puzzle)
    q_table = import_q_table(os.path.join("gui", "puzzles", puzzle_name, "Q_table.txt"))
    (states, actions) = prepare_data(action_map, q_table)
    train_nn(model, states, actions, epochs=epochs, batch_size=batch_size)
    model.save(puzzle_name+".nn")
    return model, puzzle, action_map

In [4]:
def nn_get_greedy_move (model, state, action_map):
    predictions = list(model.predict([tuple(state)], use_multiprocessing=True)[0])
    max_value = None
    moves = []
    for move, value in enumerate(predictions):
        if (not max_value) or value > max_value:
            moves = [move]
            max_value = value
        elif value == max_value:
            moves.append(move)
    return action_map[random.choice(moves)]

def nn_get_epsilon_greedy_move (model, state, action_map, epsilon=0.5):
    if random.random() > epsilon:
        return nn_get_greedy_move(model, state, action_map)
    else:
        return random.choice(action_map)

def nn_solve_puzzle (model, state, solved_state, puzzle, action_map, max_moves, epsilon=0):
    moves = []
    while max_moves >= 0:
        if state == solved_state:
            return moves, True
        else :
            move = nn_get_epsilon_greedy_move(model, state, action_map, epsilon=epsilon)
            moves.append(move)
            state = perform_action(state, puzzle.moves[move])
            max_moves -= 1
    return moves, False

In [5]:
## {{{ http://code.activestate.com/recipes/577058/ (r2)
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while 1:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return default
        elif choice in valid.keys():
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
                             "(or 'y' or 'n').\n")
## end of http://code.activestate.com/recipes/577058/ }}}

puzzle_name = input("Which puzzle? ")
model = None
puzzle = None
action_map = []
if query_yes_no("Load the existing neural network?"):
    puzzle = Twisty_Puzzle()
    puzzle.load_puzzle(puzzle_name)
    model = keras.models.load_model(puzzle_name+".nn")
    action_map = compute_action_map(puzzle)
else:
    model, puzzle, action_map = prepare_nn(puzzle_name, 1000)

Which puzzle? ivy_cube_2
Load the existing neural network? [Y/n] n


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

From /home/thomas/.local/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


From /home/thomas/.local/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ivy_cube_2.nn/assets


Assets written to: ivy_cube_2.nn/assets


In [6]:
puzzle.reset_to_solved()
solved_state = state_for_ai(puzzle.SOLVED_STATE)[0]
state = deepcopy(solved_state)
scramble_moves = scramble(state, puzzle.moves, max_moves=1)
start_state = deepcopy(state)

print("solved state:", solved_state)
print("start:       ", state)
(solution, solved) = nn_solve_puzzle(model, state, solved_state, puzzle, action_map, 10)
print("end:         ", state)
print(solved)
print(solution)

for move in scramble_moves:
    puzzle.perform_move(move)
for move in solution:
    puzzle.perform_move(move)

solved state: [0, 1, 2, 0, 3, 4, 5, 3, 2, 1, 4, 5, 0, 2, 3, 4, 1, 5]
start:        [2, 0, 1, 0, 3, 4, 5, 3, 2, 1, 4, 5, 2, 1, 3, 4, 0, 5]
end:          [2, 0, 1, 3, 4, 0, 5, 3, 2, 1, 4, 5, 3, 1, 4, 2, 0, 5]
False
['wrg', 'wrg', 'wrg', 'wrg', 'wrg', 'wrg', 'wrg', 'wrg', 'wrg', 'wrg', 'wrg']
